# Coverage calculations comparison with SPAdes coverage

SPAdes coverages output in the contig header ID seem to differ from the calculated coverage using `samtools` and `bedtools`. 

The general coverage calculation as described initially from `contig_coverage_from_bedtools.pl` is as follows:

coverage = sum(coverage-depth * bases w/coverage-depth) / sum(bases w/coverage-depth)



In [1]:
import plotly.graph_objects as go
from chart_studio.plotly import plot, iplot
from plotly.subplots import make_subplots
import numpy as np

In [2]:
import pandas as pd
cov = '../tests/data/comparisons.cov.tsv'
df = pd.read_csv(cov, sep='\t')
df.head()

,contig,bedtools_coverage,spades_coverage
0,NODE_1000_length_15201_cov_224.564,397.993750,224.564
1,NODE_1001_length_15189_cov_223.754,398.305221,223.754
2,NODE_1002_length_15170_cov_223.896,398.799473,223.896
3,NODE_1003_length_15161_cov_224.737,399.037794,224.737
4,NODE_1004_length_15148_cov_223.915,399.378466,223.915


In [3]:
df = df[df.spades_coverage != 'genome']
df.spades_coverage = df.spades_coverage.astype('float64')
df.head()

,contig,bedtools_coverage,spades_coverage
0,NODE_1000_length_15201_cov_224.564,397.993750,224.564
1,NODE_1001_length_15189_cov_223.754,398.305221,223.754
2,NODE_1002_length_15170_cov_223.896,398.799473,223.896
3,NODE_1003_length_15161_cov_224.737,399.037794,224.737
4,NODE_1004_length_15148_cov_223.915,399.378466,223.915


In [4]:
df = df.assign(cov_diff=lambda x: abs(x.bedtools_coverage - x.spades_coverage))
df.head()

,contig,bedtools_coverage,spades_coverage,cov_diff
0,NODE_1000_length_15201_cov_224.564,397.993750,224.564,173.429750
1,NODE_1001_length_15189_cov_223.754,398.305221,223.754,174.551221
2,NODE_1002_length_15170_cov_223.896,398.799473,223.896,174.903473
3,NODE_1003_length_15161_cov_224.737,399.037794,224.737,174.300794
4,NODE_1004_length_15148_cov_223.915,399.378466,223.915,175.463466


In [5]:
df.sort_values(by=['bedtools_coverage', 'spades_coverage'], inplace=True)
df.head(12)

,contig,bedtools_coverage,spades_coverage,cov_diff
2694,NODE_3437_length_105_cov_45.64,3.000000,45.6400,42.640000
2673,NODE_3411_length_148_cov_216.581,8.428571,216.5810,208.152429
2689,NODE_3428_length_120_cov_413.938,22.558333,413.9380,391.379667
2692,NODE_3432_length_111_cov_145.643,23.504505,145.6430,122.138495
2584,NODE_3327_length_225_cov_22.2353,30.000000,22.2353,7.764700
2669,NODE_3407_length_153_cov_625.765,33.084967,625.7650,592.680033
2675,NODE_3413_length_146_cov_110.374,36.726027,110.3740,73.647973
2527,NODE_3276_length_249_cov_350.356,41.396761,350.3560,308.959239
2691,NODE_3431_length_111_cov_234.304,41.981982,234.3040,192.322018
2693,NODE_3434_length_108_cov_453.679,45.518519,453.6790,408.160481


In [6]:
bedcovs = df.bedtools_coverage.tolist()
spadescovs = df.spades_coverage.tolist()
cov_diffs = df.cov_diff.tolist()
display(df['cov_diff'].describe())

count    3425.000000
mean      165.430325
std       103.433452
min         0.209329
25%       155.523854
50%       171.699454
75%       174.909931
max      2608.847161
Name: cov_diff, dtype: float64

In [10]:
# fig = go.Figure()
fig = make_subplots(rows=2)

In [11]:
fig.add_trace(go.Scatter(y=bedcovs, name='bedtools coverage'), row=1, col=1)
fig.add_trace(go.Scatter(y=spadescovs, name='SPAdes k-mer coverage'), row=1, col=1)
fig.add_trace(go.Scatter(y=cov_diffs, name='coverage difference'), row=2, col=1)
fig.show()